In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?partNum=' + input_.loc[a, 'Part_Number'] + '&brand=' + input_.loc[a, 'Brand'] + '&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'JOIN_PNB': input_.loc[a, 'JOIN_PNB'],
                                         'Src': '' if len(json_data) == 0 else json_data[0]['image_BG_Url']}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Brand': input_.loc[a, 'Brand']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'JOIN_PNB'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(50):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_PNB'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./src-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number', 'Brand'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./src_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1141

[ok] - AX27;STD
[尝试次数：1] - [剩余数量：1091] - [当前时间：14:03:39]

[ok] - AX42;STI
[尝试次数：1] - [剩余数量：1090] - [当前时间：14:03:39]

[ok] - AS432;STD
[尝试次数：1] - [剩余数量：1089] - [当前时间：14:03:39]

[ok] - AX35;STD
[尝试次数：1] - [剩余数量：1088] - [当前时间：14:03:39]

[ok] - ATS24;STD
[尝试次数：1] - [剩余数量：1087] - [当前时间：14:03:39]

[ok] - AX25;STD
[尝试次数：1] - [剩余数量：1086] - [当前时间：14:03:39]

[ok] - ATS35;STI
[尝试次数：1] - [剩余数量：1085] - [当前时间：14:03:39]

[ok] - ATS23;STI
[尝试次数：1] - [剩余数量：1084] - [当前时间：14:03:39]

[ok] - ATS32;STI
[尝试次数：1] - [剩余数量：1083] - [当前时间：14:03:39]

[ok] - AX11;STI
[尝试次数：1] - [剩余数量：1082] - [当前时间：14:03:39]

[ok] - AX1T;TTR
[尝试次数：1] - [剩余数量：1081] - [当前时间：14:03:39]

[ok] - AX43;STI
[尝试次数：1] - [剩余数量：1080] - [当前时间：14:03:39]

[ok] - AX2;STD
[尝试次数：1] - [剩余数量：1079] - [当前时间：14:03:39]

[ok] - AX45;STI
[尝试次数：1] - [剩余数量：1078] - [当前时间：14:03:39]

[ok] - AX3;STD
[尝试次数：1] - [剩余数量：1077] - [当前时间：14:03:39]

[ok] - AS185;STI
[尝试次数：1] - [剩余数量：1076] - [当前时间：14:03:39]

[ok] - AX8;STD
[尝试次数：1] - [剩余数量：1075] - [当前时间：14:03:39]

[

[ok] - AX193;STI
[尝试次数：11] - [剩余数量：951] - [当前时间：14:03:58]

[ok] - AX212;STI
[尝试次数：5] - [剩余数量：950] - [当前时间：14:03:59]

[ok] - AX222;STI
[尝试次数：1] - [剩余数量：949] - [当前时间：14:03:59]

[ok] - AX-198;STI
[尝试次数：5] - [剩余数量：948] - [当前时间：14:03:59]

[ok] - AX229;STD
[尝试次数：1] - [剩余数量：947] - [当前时间：14:03:59]

[ok] - AX140;STD
[尝试次数：35] - [剩余数量：946] - [当前时间：14:03:59]

[ok] - AX208;STI
[尝试次数：2] - [剩余数量：945] - [当前时间：14:03:59]

[ok] - AX232;STD
[尝试次数：1] - [剩余数量：944] - [当前时间：14:04:00]

[ok] - AX214;STD
[尝试次数：7] - [剩余数量：943] - [当前时间：14:04:00]

[ok] - AX188;STI
[尝试次数：20] - [剩余数量：942] - [当前时间：14:04:00]

[ok] - AX224;STI
[尝试次数：3] - [剩余数量：941] - [当前时间：14:04:00]

[ok] - AX241;STD
[尝试次数：1] - [剩余数量：940] - [当前时间：14:04:01]

[ok] - AX231;STI
[尝试次数：3] - [剩余数量：939] - [当前时间：14:04:01]

[ok] - AX219;STI
[尝试次数：7] - [剩余数量：938] - [当前时间：14:04:01]

[ok] - AX240;STD
[尝试次数：3] - [剩余数量：937] - [当前时间：14:04:01]

[ok] - AX236;STI
[尝试次数：6] - [剩余数量：936] - [当前时间：14:04:02]

[ok] - AX199;STI
[尝试次数：17] - [剩余数量：935] - [当前时间：14:04:02]

[ok] - AX

[ok] - AX373;STD
[尝试次数：1] - [剩余数量：810] - [当前时间：14:04:31]

[ok] - AX392;STI
[尝试次数：1] - [剩余数量：809] - [当前时间：14:04:31]

[ok] - AX393;STI
[尝试次数：1] - [剩余数量：808] - [当前时间：14:04:32]

[ok] - AX383;STI
[尝试次数：1] - [剩余数量：807] - [当前时间：14:04:32]

[ok] - AX377;STI
[尝试次数：1] - [剩余数量：806] - [当前时间：14:04:32]

[ok] - AX397;STI
[尝试次数：1] - [剩余数量：805] - [当前时间：14:04:32]

[ok] - AX390;STI
[尝试次数：1] - [剩余数量：804] - [当前时间：14:04:33]

[ok] - AX395;STI
[尝试次数：1] - [剩余数量：803] - [当前时间：14:04:33]

[ok] - AX391;STI
[尝试次数：1] - [剩余数量：802] - [当前时间：14:04:33]

[ok] - AX399;STI
[尝试次数：1] - [剩余数量：801] - [当前时间：14:04:33]

[ok] - AX402;STI
[尝试次数：1] - [剩余数量：800] - [当前时间：14:04:33]

[ok] - AX404;STI
[尝试次数：1] - [剩余数量：799] - [当前时间：14:04:33]

[ok] - AX372;STD
[尝试次数：1] - [剩余数量：798] - [当前时间：14:04:33]

[ok] - AX408;STD
[尝试次数：1] - [剩余数量：797] - [当前时间：14:04:34]

[ok] - AX405;STI
[尝试次数：1] - [剩余数量：796] - [当前时间：14:04:34]

[ok] - AX406;STI
[尝试次数：1] - [剩余数量：795] - [当前时间：14:04:34]

[ok] - AX411;STI
[尝试次数：1] - [剩余数量：794] - [当前时间：14:04:34]

[ok] - AX400;S

[ok] - AX100;STI
[尝试次数：8] - [剩余数量：664] - [当前时间：14:04:59]

[ok] - AX109;STD
[尝试次数：4] - [剩余数量：663] - [当前时间：14:04:59]

[ok] - ETS5;STI
[尝试次数：1] - [剩余数量：662] - [当前时间：14:04:59]

[ok] - ETS7;STI
[尝试次数：1] - [剩余数量：661] - [当前时间：14:04:59]

[ok] - AX118;STD
[尝试次数：5] - [剩余数量：660] - [当前时间：14:04:59]

[ok] - AX644;STD
[尝试次数：1] - [剩余数量：659] - [当前时间：14:04:59]

[ok] - AX86;STD
[尝试次数：10] - [剩余数量：658] - [当前时间：14:04:59]

[ok] - ETS10;STI
[尝试次数：1] - [剩余数量：657] - [当前时间：14:04:59]

[ok] - ETS13;STI
[尝试次数：1] - [剩余数量：656] - [当前时间：14:04:59]

[ok] - ETS12;STI
[尝试次数：1] - [剩余数量：655] - [当前时间：14:04:59]

[ok] - AX633;STI
[尝试次数：1] - [剩余数量：654] - [当前时间：14:04:59]

[ok] - ETS21;STI
[尝试次数：1] - [剩余数量：653] - [当前时间：14:04:59]

[ok] - ETS22;STI
[尝试次数：1] - [剩余数量：652] - [当前时间：14:04:59]

[ok] - ETS9;STD
[尝试次数：1] - [剩余数量：651] - [当前时间：14:04:59]

[ok] - ETS37;STI
[尝试次数：2] - [剩余数量：650] - [当前时间：14:05:00]

[ok] - ETS25;STI
[尝试次数：1] - [剩余数量：649] - [当前时间：14:05:00]

[ok] - AX54;STI
[尝试次数：12] - [剩余数量：648] - [当前时间：14:05:00]

[ok] - ETS44;STI


[ok] - ETS195;STI
[尝试次数：5] - [剩余数量：522] - [当前时间：14:05:25]

[ok] - ETS168;STD
[尝试次数：18] - [剩余数量：521] - [当前时间：14:05:26]

[ok] - ETS176;STD
[尝试次数：5] - [剩余数量：520] - [当前时间：14:05:26]

[ok] - ETS184;STI
[尝试次数：10] - [剩余数量：519] - [当前时间：14:05:26]

[ok] - ETS132;STI
[尝试次数：38] - [剩余数量：518] - [当前时间：14:05:26]

[ok] - ETS178;STD
[尝试次数：12] - [剩余数量：517] - [当前时间：14:05:27]

[ok] - ETS174;STI
[尝试次数：5] - [剩余数量：516] - [当前时间：14:05:27]

[ok] - ETS185;STI
[尝试次数：11] - [剩余数量：515] - [当前时间：14:05:28]

[ok] - ETS190;STI
[尝试次数：9] - [剩余数量：514] - [当前时间：14:05:28]

[ok] - ETS213;STI
[尝试次数：2] - [剩余数量：513] - [当前时间：14:05:28]

[ok] - ETS203;STI
[尝试次数：2] - [剩余数量：512] - [当前时间：14:05:28]

[ok] - ETS205;STI
[尝试次数：4] - [剩余数量：511] - [当前时间：14:05:29]

[ok] - ETS196;STI
[尝试次数：11] - [剩余数量：510] - [当前时间：14:05:29]

[ok] - ETS217;STI
[尝试次数：3] - [剩余数量：509] - [当前时间：14:05:29]

[ok] - ETS204;STI
[尝试次数：4] - [剩余数量：508] - [当前时间：14:05:29]

[ok] - ETS193;STI
[尝试次数：8] - [剩余数量：507] - [当前时间：14:05:29]

[ok] - ETS224;STD
[尝试次数：1] - [剩余数量：506] - [当前时间：14

[ok] - S2860;STI
[尝试次数：1] - [剩余数量：382] - [当前时间：14:06:05]

[ok] - S2945;STI
[尝试次数：1] - [剩余数量：381] - [当前时间：14:06:05]

[ok] - S2478;STD
[尝试次数：1] - [剩余数量：380] - [当前时间：14:06:06]

[ok] - S3042;STD
[尝试次数：1] - [剩余数量：379] - [当前时间：14:06:06]

[ok] - S2907;STI
[尝试次数：1] - [剩余数量：378] - [当前时间：14:06:07]

[ok] - S2522;STI
[尝试次数：1] - [剩余数量：377] - [当前时间：14:06:07]

[ok] - S2964;STI
[尝试次数：1] - [剩余数量：376] - [当前时间：14:06:07]

[ok] - TS-6;STD
[尝试次数：1] - [剩余数量：375] - [当前时间：14:06:07]

[ok] - S3064;STD
[尝试次数：1] - [剩余数量：374] - [当前时间：14:06:08]

[ok] - TS-4;STD
[尝试次数：1] - [剩余数量：373] - [当前时间：14:06:08]

[ok] - TS6T;TTR
[尝试次数：1] - [剩余数量：372] - [当前时间：14:06:08]

[ok] - S2894;STD
[尝试次数：1] - [剩余数量：371] - [当前时间：14:06:08]

[ok] - TS24T;TTR
[尝试次数：1] - [剩余数量：370] - [当前时间：14:06:09]

[ok] - S2936;STD
[尝试次数：1] - [剩余数量：369] - [当前时间：14:06:09]

[ok] - SC706;STI
[尝试次数：1] - [剩余数量：368] - [当前时间：14:06:09]

[ok] - AX632;STI
[尝试次数：2] - [剩余数量：367] - [当前时间：14:06:10]

[ok] - TS-57;STI
[尝试次数：1] - [剩余数量：366] - [当前时间：14:06:10]

[ok] - S2969;STD


[ok] - TX21;STI
[尝试次数：1] - [剩余数量：238] - [当前时间：14:06:34]

[ok] - TX48;STD
[尝试次数：1] - [剩余数量：237] - [当前时间：14:06:34]

[ok] - TS-422;STD
[尝试次数：1] - [剩余数量：236] - [当前时间：14:06:35]

[ok] - TX37T;TTR
[尝试次数：1] - [剩余数量：235] - [当前时间：14:06:35]

[ok] - TX55;STI
[尝试次数：1] - [剩余数量：234] - [当前时间：14:06:35]

[ok] - TX54;STI
[尝试次数：1] - [剩余数量：233] - [当前时间：14:06:35]

[ok] - TX56;STI
[尝试次数：1] - [剩余数量：232] - [当前时间：14:06:35]

[ok] - TX64;STD
[尝试次数：1] - [剩余数量：231] - [当前时间：14:06:35]

[ok] - TX40;STI
[尝试次数：1] - [剩余数量：230] - [当前时间：14:06:35]

[ok] - TX58;STI
[尝试次数：1] - [剩余数量：229] - [当前时间：14:06:35]

[ok] - TX60;STI
[尝试次数：1] - [剩余数量：228] - [当前时间：14:06:35]

[ok] - TX66;STD
[尝试次数：1] - [剩余数量：227] - [当前时间：14:06:35]

[ok] - TX37;STI
[尝试次数：1] - [剩余数量：226] - [当前时间：14:06:35]

[ok] - TX68;STI
[尝试次数：1] - [剩余数量：225] - [当前时间：14:06:36]

[ok] - TX31;STI
[尝试次数：1] - [剩余数量：224] - [当前时间：14:06:36]

[ok] - AX239;STI
[尝试次数：7] - [剩余数量：223] - [当前时间：14:06:36]

[ok] - TX73T;TTR
[尝试次数：1] - [剩余数量：222] - [当前时间：14:06:36]

[ok] - TX6;STD
[尝试次数：1] - 

[ok] - TX246;STI
[尝试次数：3] - [剩余数量：96] - [当前时间：14:06:57]

[ok] - TX142;STI
[尝试次数：35] - [剩余数量：95] - [当前时间：14:06:57]

[ok] - TX228;STI
[尝试次数：1] - [剩余数量：94] - [当前时间：14:06:57]

[ok] - TX253;STI
[尝试次数：5] - [剩余数量：93] - [当前时间：14:06:57]

[ok] - ETS314;STD
[尝试次数：21] - [剩余数量：92] - [当前时间：14:06:57]

[ok] - TX231;STD
[尝试次数：13] - [剩余数量：91] - [当前时间：14:06:58]

[ok] - TX258;STI
[尝试次数：2] - [剩余数量：90] - [当前时间：14:06:58]

[ok] - TX261;STD
[尝试次数：1] - [剩余数量：89] - [当前时间：14:06:58]

[ok] - TX233;STI
[尝试次数：15] - [剩余数量：88] - [当前时间：14:06:59]

[ok] - TX265;STD
[尝试次数：2] - [剩余数量：87] - [当前时间：14:06:59]

[ok] - TX255;STI
[尝试次数：8] - [剩余数量：86] - [当前时间：14:06:59]

[ok] - TX179;STD
[尝试次数：29] - [剩余数量：85] - [当前时间：14:06:59]

[ok] - TX252;STI
[尝试次数：8] - [剩余数量：84] - [当前时间：14:06:59]

[ok] - TX260;STD
[尝试次数：6] - [剩余数量：83] - [当前时间：14:07:00]

[ok] - TX273;STI
[尝试次数：1] - [剩余数量：82] - [当前时间：14:07:00]

[ok] - TX229;STD
[尝试次数：18] - [剩余数量：81] - [当前时间：14:07:00]

[ok] - TX267;STD
[尝试次数：2] - [剩余数量：80] - [当前时间：14:07:00]

[ok] - TX254;STD
[尝试次数：7

[ok] - TX326;STD
[尝试次数：3] - [剩余数量：0] - [当前时间：14:08:25]

[ok] - AX141;STI
[尝试次数：37] - [剩余数量：0] - [当前时间：14:08:28]

输出ing...

Done ~
